# A CNN-LSTM framework for the solar wind density forecasting
## ConvLSTM training
In this notebook we train a ConvLSTM network to predict solar wind densities (electrons + protons)


#### Notebook Contributors
* Andrea Giuseppe Di Francesco -- email: difrancesco.1836928@studenti.uniroma1.it
* Massimo Coppotelli -- email: coppotelli.1705325@studenti.uniroma1.it

In [1]:
#!pip install pandas
# !pip install numpy
# !pip install torch
# !pip install matplotlib
# !pip install torchvision
!pip install wandb

  Using cached wandb-0.13.7-py2.py3-none-any.whl (1.9 MB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 407.1 kB/s eta 0:00:00a 0:00:01
  Using cached shortuuid-1.0.11-py3-none-any.whl (10 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached GitPython-3.1.30-py3-none-any.whl (184 kB)
  Using cached protobuf-4.21.12-cp37-abi3-manylinux2014_x86_64.whl (409 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 1.1 MB/s eta 0:00:0000:0100:01
  Using cached sentry_sdk-1.12.1-py2.py3-none-any.whl (174 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=039ff43d3d0fd4c1fe528de9bff9e31a39b8430aaed060062648ce8d66967267
  Stored in dire

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image
import wandb
from convlstm import *
from utils import *
from init import *

In [2]:
if wb:
    wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: coppotelli-1705325 (small_sunbirds). Use `wandb login --relogin` to force relogin


In [6]:
wind_dataset = pd.read_csv('./datasets/wind_dataset_1d_res.xlsx', index_col = 0)
sun_dataset = load_data('./datasets/ARI_image_dataset.json')

JSONDecodeError: Expecting ',' delimiter: line 1 column 89279971 (char 89279970)

In [ ]:
import json

# jfile = open('./datasets/ARI_image_dataset.json', "r")
dicti = 
image =

In [2]:
image = torch.randn((1,4,1,224,224))

model = ConvLSTM(n_channels, n_hidden_channels, kernel_size, n_layers, batch_first, bias)


In [3]:
output = model(image)